In [1]:
cd APTOS_Detection/

/home/shashank/All_code/Research/APTOS_Detection


In [2]:
import torch
from torch import nn, optim
import os
import config
import numpy as np
import random
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import cohen_kappa_score
from efficientnet_pytorch import EfficientNet
from dataset import DRDataset
import datetime
from torchvision.utils import save_image
from earlystopper import EarlyStopper
import csv 
from utils import (
    load_checkpoint,
    save_checkpoint,
    check_accuracy,
    make_prediction,
    get_csv_for_blend,
)

In [ ]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")


# Set a specific seed value (you can use any integer)
seed = 42
set_seed(seed)

Random seed set as 42


In [ ]:
def train_one_epoch(loader, model, optimizer, loss_fn, scaler, device):
    losses = []
    loop = tqdm(loader)
    for batch_idx, (data, targets, _) in enumerate(loop):
        # save examples and make sure they look ok with the data augmentation,
        # tip is to first set mean=[0,0,0], std=[1,1,1] so they look "normal"
        #save_image(data, f"hi_{batch_idx}.png")

        data = data.to(device=device)
        #print(data.shape)
        #print("Targets",targets)
        targets = targets.to(device=device)

        # forward
        with torch.cuda.amp.autocast():
            scores = model(data)
            loss = loss_fn(scores, targets.unsqueeze(1).float())

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        loop.set_postfix(loss=loss.item())

    print(f"Loss average over epoch: {sum(losses)/len(losses)}")

In [ ]:
hyperparameter_combinations = [
{'batch_size': 64, 'learning_rate': 3e-05, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 0.0002, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 2e-05, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 0.005, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 0.0005, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 5e-05, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 0.003, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 0.0003, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 3e-05, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 0.001, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 0.0001, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 1e-05, 'epochs': 700},
# {'batch_size': 32, 'learning_rate': 0.002, 'epochs': 700},
# {'batch_size': 32, 'learning_rate': 0.0002, 'epochs': 700},
# {'batch_size': 32, 'learning_rate': 2e-05, 'epochs': 700},
# {'batch_size': 32, 'learning_rate': 0.005, 'epochs': 700},
# {'batch_size': 32, 'learning_rate': 0.0005, 'epochs': 700},
# {'batch_size': 32, 'learning_rate': 5e-05, 'epochs': 700},
# {'batch_size': 32, 'learning_rate': 0.003, 'epochs': 700},
# {'batch_size': 32, 'learning_rate': 0.0003, 'epochs': 700},
# {'batch_size': 32, 'learning_rate': 3e-05, 'epochs': 700},
# {'batch_size': 32, 'learning_rate': 0.001, 'epochs': 700},
# {'batch_size': 32, 'learning_rate': 0.0001, 'epochs': 700},
# {'batch_size': 32, 'learning_rate': 1e-05, 'epochs': 700},
]
#131,64,3e-05,0,82.90636042402826,78.10476751030018,0

In [ ]:
def training_main(hyperparams,early_stopper):
    BATCH_SIZE = hyperparams['batch_size']
    LEARNING_RATE = hyperparams['learning_rate']
    NUM_EPOCHS = hyperparams['epochs']
    
    best_accuracy = 0.0
    best_precision = 0.0
    best_recall = 0.0
    best_train_accuracy = 0.0
    best_f1 = 0.0
    best_train_precision = 0
    best_train_recall = 0
        
    current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_folder = f"output_results/training_{current_time}_{BATCH_SIZE}_{LEARNING_RATE}"
    os.makedirs(output_folder)
    

    
    train_ds = DRDataset(
        images_folder="/home/shashank/All_code/Research/APTOS_dataset/aptos2019_blindness_detection/Real_Synthetic/RS_train_split",
        path_to_csv="/home/shashank/All_code/Research/APTOS_dataset/aptos2019_blindness_detection/Real_Synthetic/RS_train_split.csv",
        transform=config.val_transforms,
    )
    val_ds = DRDataset(
        images_folder="/home/shashank/All_code/Research/APTOS_dataset/aptos2019_blindness_detection/Real_Synthetic/RS_val_split",
        path_to_csv="/home/shashank/All_code/Research/APTOS_dataset/aptos2019_blindness_detection/Real_Synthetic/RS_val_split.csv",
        transform=config.val_transforms,
    )
    test_ds = DRDataset(
        images_folder="/home/shashank/All_code/test/images_resized_192",
        path_to_csv="/home/shashank/All_code/Research/APTOS_dataset/aptos2019_blindness_detection/test.csv",
        transform=config.val_transforms,
        train=False,
    )
    test_loader = DataLoader(
        test_ds, batch_size=BATCH_SIZE, num_workers=6, shuffle=False
    )
    train_loader = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
        pin_memory=config.PIN_MEMORY,
        shuffle=False,
    )
   
    val_loader = DataLoader(
        val_ds,
        batch_size=BATCH_SIZE,
        num_workers=config.NUM_WORKERS, 
        pin_memory=config.PIN_MEMORY,
        shuffle=False,
    )
    
    loss_fn = nn.MSELoss()
    
    model = EfficientNet.from_pretrained("efficientnet-b7")
    #model = EfficientNet.from_name("efficientnet-b6")
    model._fc = nn.Linear(2560, 1)
    model = model.to(config.DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=config.WEIGHT_DECAY)
    scaler = torch.cuda.amp.GradScaler()

    print("OS Path ",os.listdir())    
    if config.LOAD_MODEL and config.CHECKPOINT_FILE in os.listdir():
        load_checkpoint(torch.load(config.CHECKPOINT_FILE), model, optimizer,LEARNING_RATE)
        print("Model Loaded !!!!!!!!")

    #model._fc = nn.Linear(1536,1)
        
    # Run after training is done and you've achieved good result
    # on validation set, then run train_blend.py file to use information
    # about both eyes concatenated
    #get_csv_for_blend(val_loader, model, "../train/val_blend.csv")
    #get_csv_for_blend(train_loader, model, "../train/train_blend.csv")
    #get_csv_for_blend(test_loader, model, "../train/test_blend.csv")
    #make_prediction(model, test_loader, "submission.csv")
    #import sys
    #sys.exit()
    #make_prediction(model, test_loader)

    for epoch in range(NUM_EPOCHS):
        print("Epoch - ", epoch)
        train_one_epoch(train_loader, model, optimizer, loss_fn, scaler, config.DEVICE)

        # get on validation
        preds, labels,val_accuracy, val_precision, val_recall, val_f1 = check_accuracy(val_loader, model, config.DEVICE)
        quad_score_val = cohen_kappa_score(labels, preds, weights='quadratic')
        # print(f"QuadraticWeightedKappa (Validation): {cohen_kappa_score(labels, preds, weights='quadratic')}")
        print(f"QuadraticWeightedKappa (Validation): ", quad_score_val)
        
        # get on train
        preds, labels,train_accuracy,train_precision,train_recall, train_f1 = check_accuracy(train_loader, model, config.DEVICE)
        quad_score_train = cohen_kappa_score(labels, preds, weights='quadratic')
        print(f"QuadraticWeightedKappa (Training): {cohen_kappa_score(labels, preds, weights='quadratic')}")
        if val_accuracy > best_accuracy:
            best_train_accuracy = train_accuracy
            best_accuracy = val_accuracy
            epoch_checkpoint = epoch+1
            
        if val_precision > best_precision:
            #best_train_precision = train_precision
            best_precision = val_precision
            
        if val_recall > best_recall:
            #best_train_recall = train_recall
            best_recall = val_recall
            
        if val_f1 > best_f1:
            best_f1 = val_f1
            print()

           
        #if config.SAVE_MODEL and epoch % 10 == 0 :
            if config.SAVE_MODEL:
                checkpoint = {
                    "state_dict": model.state_dict(),
                    "optimizer": optimizer.state_dict(),
                }
            #save_checkpoint(checkpoint, filename=f"/home/shashank/All_code/Research/APTOS_Detection/weights/b3_{epoch}.pth.tar")
            save_checkpoint(checkpoint, filename=os.path.join(output_folder+"/b3_.pth.tar"))

        
        if early_stopper.early_stop(val_accuracy):
            
            print("Early stopping triggered!")
            break
    return epoch, best_accuracy, best_train_accuracy, best_precision, best_recall, best_f1, quad_score_val        

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
log_file = os.path.join("/home/shashank/All_code/Research/APTOS_Detection/output_results", f"{current_time}_ENb7_log.csv")
with open(log_file, 'w', newline='') as csvfile:
    fieldnames = ['epoch','batch_size','Learning_rate', 'training_loss', 'training_accuracy',  'val_accuracy', 'val_loss','val_precision','val_recall','val_f1','Qudratic_kappa_score_val']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

for hyperparams in hyperparameter_combinations:
    torch.cuda.empty_cache()
    # Instantiate EarlyStopper with desired patience and min_delta
    early_stopper = EarlyStopper(patience=20, min_delta=0.001)
    print("hyperparameter_combinations >>>>>>>>>>>>>> ",hyperparams)
    epoch , best_accuracy,best_train_accuracy, best_precision, best_recall, best_f1,quad_score_val = training_main(hyperparams,early_stopper)
    

    with open(log_file, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({
            'epoch': epoch,
            'batch_size': hyperparams['batch_size'],
            'Learning_rate': hyperparams['learning_rate'],
            'training_loss': "0",
            'training_accuracy': best_train_accuracy,
            'val_accuracy': best_accuracy,
            'val_loss' : "0",
            'val_precision' : best_precision,
            'val_recall' : best_recall,
            'val_f1' : best_f1,
            'Qudratic_kappa_score_val': quad_score_val,
            
                })

hyperparameter_combinations >>>>>>>>>>>>>>  {'batch_size': 64, 'learning_rate': 3e-05, 'epochs': 1}
Loaded pretrained weights for efficientnet-b7
OS Path  ['submission.csv', 'output_results', 'weights', 'val1csv.csv', '__pycache__', 'train_blend.py', 'fidelity', 'config.py', 'preprocess_images.py', 'earlystopper.py', 'utils.py', 'train1', 'train.py', 'train.ipynb', 'val1', 'dataset.py', 'train1csv.csv']
Epoch -  0


100%|██████████| 107/107 [00:27<00:00,  3.93it/s, loss=1.44] 


Loss average over epoch: 4.6927641858564355


100%|██████████| 27/27 [00:04<00:00,  6.09it/s]
/home/shashank/anaconda3/envs/Cuda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Got 521 / 1699 with accuracy 30.67
Shapes - all_preds: (1699,) all_labels: (1699,)
DataType - all_preds: int64 all_labels: int64
Unique labels: [0 1 2 3 4]
Accuracy: 30.67
Precision: 0.35
Recall: 0.31
F1 Score: 0.20
QuadraticWeightedKappa (Validation):  0.1951168651204248


...

Got 652 / 1699 with accuracy 38.38
Shapes - all_preds: (1699,) all_labels: (1699,)
DataType - all_preds: int64 all_labels: int64
Unique labels: [0 1 2 3 4]
Accuracy: 38.38
Precision: 0.25
Recall: 0.38
F1 Score: 0.30
QuadraticWeightedKappa (Validation):  0.6116447378584242


100%|██████████| 107/107 [00:16<00:00,  6.54it/s]


Got 2520 / 6792 with accuracy 37.10
Shapes - all_preds: (6792,) all_labels: (6792,)
DataType - all_preds: int64 all_labels: int64
Unique labels: [0 1 2 3 4]
Accuracy: 37.10
Precision: 0.45
Recall: 0.37
F1 Score: 0.28
QuadraticWeightedKappa (Training): 0.6299566799889269

=> Saving checkpoint
hyperparameter_combinations >>>>>>>>>>>>>>  {'batch_size': 64, 'learning_rate': 2e-05, 'epochs': 700}
Loaded pretrained weights for efficientnet-b7
OS Path  ['submission.csv', 'output_results', 'weights', 'val1csv.csv', '__pycache__', 'train_blend.py', 'fidelity', 'config.py', 'preprocess_images.py', 'earlystopper.py', 'utils.py', 'train1', 'train.py', 'train.ipynb', 'val1', 'dataset.py', 'train1csv.csv']
Epoch -  0


 20%|█▉        | 21/107 [00:05<00:23,  3.71it/s, loss=8.7] 


KeyboardInterrupt: 